![Banner logo](https://raw.githubusercontent.com/CitrineInformatics/community-tools/master/templates/fig/citrine_banner_2.png "Banner logo")

## Sequential Learning Challenge

*Authors: Zach del Rosario (zdelrosario@citrine.io)*

Now that we've "eaten our vegetables", we're ready to start using machine learning to study materials science problems. We'll use the [Agrawal et al. (2014) IMMI](https://citrination.com/datasets/150670/show_search?searchMatchOption=fuzzyMatch) dataset to study the relationship between alloy composition and fatigue strength.

### Learning outcomes
By working through this notebook, you will be able to:

* Understand *featurization*
* Featurize inorganic materials data
* Understand *sequential learning*
* Design a machine learning model to support sequential learning in the support of designing novel materials

Tips:

In [ ]:
# Setup
import numpy as np
import pandas as pd

# Model training tools
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer

# For jupyter-matplotlib compatibility
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Custom tools
from workshop_utils import formulas2df, sequentialLearningSimulator, plotHistory

# Agrawal data from previous exercise
filename_data = "./agrawal_data.csv"

# Helper functions
def nde(y_true, y_pred):
    """Non-dimensional Model Error
    """
    mse = mean_squared_error(y_true, y_pred)
    response_std = np.std(y_true)
    
    return np.sqrt(mse) / response_std

nde_score = make_scorer(nde)

## Setup: ML on Materials Data
Now that we have learned some (but not _all_!) concepts about training machine learning models, we are finally ready to apply ML to a materials problem.

### Q1: Load the Fatigue Strength data
Load the Agrawal fatigue strength data from the earlier exercise. Its filename is stored in the variable `filename_data`. Make sure to load the data to a Pandas DataFrame, and name it `df_data`.

In [ ]:
## TASK: Load the Agrawal data from the file at filename_data
# solution-begin
df_data = pd.read_csv(filename_data)
# solution-end
df_data.head()

### Simple featurization
Alloy composition is encoded in a string in the column `chemical_formula`.

In [ ]:
df_data[['chemical_formula']].head()

Fitting a linear regression _directly_ to this _string representation_ is not feasible -- these are not continuous values! Instead, we will _featurize_ the chemical formulas by representing each element fraction as a separate column.

Note that doing this _programmatically_ is a bit tricky (it requires [regular expressions](https://en.wikipedia.org/wiki/Regular_expression)) -- I've written a simple parser to do this in a single function call. Feel free to inspect the code in `workshop_utils.py` if you'd like to see how this works! 


In [ ]:
df_composition = formulas2df(df_data['chemical_formula'])
X_compositions = df_composition.values  # Inputs for model
Y_fatigue = df_data['Fatigue Strength'] # Response (to predict)

df_composition.head()

In [ ]:
X_compositions.shape

Now we have ten physical features on which to fit our model.

### Q2: Fit a model on alloy composition


In [ ]:
## TASK: Provide featurized data in the variable X_data. Choose how to featurize the data.
# solution-begin
# I choose to cross-validate the model choices, in order to select an optimal order for the model.
Ord_all = [0,1,2,3]
n_cv = 5
NDE_cv_test_all = np.zeros((len(Ord_all), n_cv))
NDE_cv_train_all = np.zeros((len(Ord_all), n_cv))

plt.figure()

for i in range(len(Ord_all)):
    # Fit model
    poly   = PolynomialFeatures(Ord_all[i])
    X_poly = poly.fit_transform(X_compositions)
    reg    = LinearRegression().fit(X_poly, Y_fatigue)
    
    # Cross-validate
    scores = cross_validate(
        reg, poly.fit_transform(X_compositions), Y_fatigue,
        cv = n_cv,
        scoring = nde_score,
        return_train_score = True
    )
    NDE_cv_test_all[i] = scores['test_score']
    NDE_cv_train_all[i] = scores['train_score']
    # Plot all CV test instances
    plt.plot([Ord_all[i]] * n_cv, NDE_cv_test_all[i], 'k.')
NDE_cv_test = np.mean(NDE_cv_test_all, axis = 1)
NDE_cv_train = np.mean(NDE_cv_train_all, axis = 1)

plt.plot(Ord_all, NDE_cv_test, label = 'Test')
plt.legend(loc = 0)
plt.yscale('log')
plt.xlabel('Poly Order')
plt.ylabel('ND Error')
plt.show()

ind_min = np.argmin(NDE_cv_test)
print("ord_min = {}".format(Ord_all[ind_min]))
print("NDE_min = {}".format(NDE_cv_test[ind_min]))

# Based on these results, I choose order = 2, and provide the featurized data
poly   = PolynomialFeatures(2)
X_data = poly.fit_transform(X_compositions)
# solution-end

Once you complete __Q2__ by further featurizing the data, you can provide this information to the following *sequential learning simulator*.

## Sequential Learning Simulation

In [ ]:
acq_history = sequentialLearningSimulator(X_data, Y_fatigue)
plotHistory(acq_history, Y_fatigue)
plt.xlabel("Iteration")
plt.ylabel("Fatigue Strength")

### Advanced featurization
The featurization above is fairly simple; we can actually bring in *much more* information about materials by leveraging our physical insight. The [Matminer](https://hackingmaterials.lbl.gov/matminer/) package is a set of tools for data-mining on chemicals data. Their library provides tools to produce _descriptors_ (features) based on chemical compositions. The following (Optional!) code demonstrates how to use Matminer to generate numerous features based on inorganic chemical labels (compositions). The following code demonstrates how to featurize chemical data using Matminer.

In [ ]:
## Setup
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from pymatgen import Composition

def get_composition(c):
    """Attempt to parse a composition, return None if failed."""
    try:
        return Composition(c)
    except:
        return None
    
## Wrangle the composition strings
df_data['composition'] = df_data['chemical_formula'].apply(get_composition)

## Select features to compute
featurizer = MultipleFeaturizer([
    cf.Stoichiometry(),
    cf.ElementProperty.from_preset("magpie"),
    cf.ValenceOrbital(props = ['avg']),
    cf.IonProperty(fast = True)
])

## Run the featurizer
X_matminer_features = np.array(featurizer.featurize_many(df_data['composition']))
print("Featurization complete")
df_matminer_features = pd.DataFrame(
    data = X_matminer_features,
    columns = featurizer.feature_labels()
)
df_matminer_features.head() 

## More Considerations

__Feature Selection__: One 'hyperparameter' we have not explicitly mentioned is _the choice of features_ for the model. While additional features do provide more information, they also increase the number of parameters (internal coefficients) the model needs to learn. This can lead to overfitting in the same way we saw above. To help combat this issue, data scientists will perform [feature selection](https://en.wikipedia.org/wiki/Feature_selection) to choose the "most informative" features from a set, which can improve model generalizability.

__Different Models__: We considered only simple polynomial models in this tutorial, but there exist many other model types. Some highlights:

* [Gaussian processes](https://en.wikipedia.org/wiki/Gaussian_process)
* [Random forests](https://en.wikipedia.org/wiki/Random_forest)
* [Neural networks](https://en.wikipedia.org/wiki/Artificial_neural_network)
